<a href="https://colab.research.google.com/github/JhonJRB/coderhouse/blob/main/Desafio_CrossValidation_Rodriguez_Figueredo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score
from sklearn import linear_model, tree, ensemble
import re # reemplazar espacios de las variables del dataset
import seaborn as sns #visualizaciones
import statsmodels.api as sm 
from sklearn import metrics # modulo para evaluar metricas del modelo
from sklearn.ensemble import RandomForestClassifier #Algoritmo para clasificacion
from sklearn.model_selection import train_test_split #separar datos para test y train
from sklearn.preprocessing import LabelEncoder # nos permite convertir las variables categorias a numeros, (Gender M/F a 0/1)

In [2]:
df=pd.read_csv('https://raw.githubusercontent.com/Gastfigue/coderhouse/main/train.csv', on_bad_lines='skip')
df.head()

,Unnamed: 0,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,0,70172,Male,Loyal Customer,13,Personal Travel,Eco Plus,460,3,4,...,5,4,3,4,4,5,5,25,18.0,neutral or dissatisfied
1,1,5047,Male,disloyal Customer,25,Business travel,Business,235,3,2,...,1,1,5,3,1,4,1,1,6.0,neutral or dissatisfied
2,2,110028,Female,Loyal Customer,26,Business travel,Business,1142,2,2,...,5,4,3,4,4,4,5,0,0.0,satisfied
3,3,24026,Female,Loyal Customer,25,Business travel,Business,562,2,5,...,2,2,5,3,1,4,2,11,9.0,neutral or dissatisfied
4,4,119299,Male,Loyal Customer,61,Business travel,Business,214,3,3,...,3,3,4,4,3,3,3,0,0.0,satisfied


In [3]:
df=df.drop(['Unnamed: 0','id'],axis=1) # Eliminar col Unnamed y id
l=[x for x in df.columns if x  in ['Gender','Customer Type','Type of Travel', 'Class', 'satisfaction']]
df[l]=df[l].astype('category') #Convertir variables tipo int a category
avg_norm_loss = df['Arrival Delay in Minutes'].astype("float").mean(axis=0) #Imputacion de datos nulos ***
df['Arrival Delay in Minutes'].replace(np.nan, avg_norm_loss, inplace=True)

In [4]:
y = df['satisfaction']

In [5]:
numeric_cols = [cname for cname in df.columns if df[cname].dtype in ['int64', 'float64']]
X = df[numeric_cols].copy()

In [6]:
from sklearn.model_selection import KFold

# Instanciar objeto
kfold = KFold()

In [7]:
kfold.split(X)

<generator object _BaseKFold.split at 0x7fbf029d1ba0>

In [8]:
for i in kfold.split(X):
  print(i)

(array([ 20781,  20782,  20783, ..., 103901, 103902, 103903]), array([    0,     1,     2, ..., 20778, 20779, 20780]))
(array([     0,      1,      2, ..., 103901, 103902, 103903]), array([20781, 20782, 20783, ..., 41559, 41560, 41561]))
(array([     0,      1,      2, ..., 103901, 103902, 103903]), array([41562, 41563, 41564, ..., 62340, 62341, 62342]))
(array([     0,      1,      2, ..., 103901, 103902, 103903]), array([62343, 62344, 62345, ..., 83121, 83122, 83123]))
(array([    0,     1,     2, ..., 83121, 83122, 83123]), array([ 83124,  83125,  83126, ..., 103901, 103902, 103903]))


In [9]:
print("Shape of input data: {} and shape of target variable: {}".format(X.shape, y.shape))

X.head() # Show first 5 training examples

Shape of input data: (103904, 18) and shape of target variable: (103904,)


,Age,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,Food and drink,Online boarding,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes
0,13,460,3,4,3,1,5,3,5,5,4,3,4,4,5,5,25,18.0
1,25,235,3,2,3,3,1,3,1,1,1,5,3,1,4,1,1,6.0
2,26,1142,2,2,2,2,5,5,5,5,4,3,4,4,4,5,0,0.0
3,25,562,2,5,5,5,2,2,2,2,2,5,3,1,4,2,11,9.0
4,61,214,3,3,3,3,4,5,5,3,3,4,4,3,3,3,0,0.0


In [10]:
# Lets split the data into 5 folds.  
# We will use this 'kf'(KFold splitting stratergy) object as input to cross_val_score() method
kf =KFold(n_splits=5, shuffle=True, random_state=42)
cnt = 1
# split()  method generate indices to split data into training and test set.
for train_index, test_index in kf.split(X, y):
    print(f'Fold:{cnt}, Train set: {len(train_index)}, Test set:{len(test_index)}')
    cnt += 1

Fold:1, Train set: 83123, Test set:20781
Fold:2, Train set: 83123, Test set:20781
Fold:3, Train set: 83123, Test set:20781
Fold:4, Train set: 83123, Test set:20781
Fold:5, Train set: 83124, Test set:20780


In [ ]:
label_enc=LabelEncoder()
label_enc.fit(y)
y=label_enc.transform(y)


In [23]:
score = cross_val_score(tree.DecisionTreeRegressor(random_state= 42), X, y, cv=kf, scoring="accuracy")
score
score.mean()

0.9225535203526543